In [3]:
# pip install sqlalchemy pyodbc pandas scikit-learn

from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy.orm import sessionmaker
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Build safe connection URL for Windows Authentication
connection_url = URL.create(
    "mssql+pyodbc",
    host="localhost",  # or "localhost\\SQLEXPRESS" for named instance
    database="Titanic",
    query={
        "driver": "ODBC Driver 17 for SQL Server",
        "trusted_connection": "yes"
    }
)

# Step 2: Create engine
engine = create_engine(connection_url)
Session = sessionmaker(bind=engine)
session = Session()

# Step 3: Load Titanic data
query = "SELECT * FROM titanic"
df = pd.read_sql(query, engine)

# Step 4: Data preprocessing
df = df.dropna(subset=['Survived', 'Pclass', 'Sex', 'Age'])
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
X = df[['Pclass', 'Sex', 'Age']]
y = df['Survived']

# Step 5: Model training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)

# Step 6: Evaluation
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Step 7: Save predictions back
results = X_test.copy()
results['Actual'] = y_test
results['Predicted'] = y_pred
results.reset_index(inplace=True)
results.to_sql('titanic_predictions', con=engine, if_exists='replace', index=False)
print("Predictions saved to SQL Server.")


Accuracy: 0.7482517482517482
              precision    recall  f1-score   support

       False       0.80      0.78      0.79        87
        True       0.67      0.70      0.68        56

    accuracy                           0.75       143
   macro avg       0.74      0.74      0.74       143
weighted avg       0.75      0.75      0.75       143

Predictions saved to SQL Server.


In [4]:
import joblib

# Save the trained model
joblib.dump(model, 'titanic_model.pkl')
print("Model saved as titanic_model.pkl")



Model saved as titanic_model.pkl


In [7]:
!pip install fastapi

  Obtaining dependency information for fastapi from https://files.pythonhosted.org/packages/53/50/b1222562c6d270fea83e9c9075b8e8600b8479150a18e4516a6138b980d1/fastapi-0.115.14-py3-none-any.whl.metadata
  Obtaining dependency information for starlette<0.47.0,>=0.40.0 from https://files.pythonhosted.org/packages/8b/0c/9d30a4ebeb6db2b25a841afbb80f6ef9a854fc3b41be131d249a977b4959/starlette-0.46.2-py3-none-any.whl.metadata
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/69/e0/552843e0d356fbb5256d21449fa957fa4eff3bbc135a74a691ee70c7c5da/typing_extensions-4.14.0-py3-none-any.whl.metadata
  Obtaining dependency information for anyio<5,>=3.6.2 from https://files.pythonhosted.org/packages/a1/ee/48ca1a7c89ffec8b6a0c5d02b89c305671d5ffd8d3c94acf8b8c408575bb/anyio-4.9.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/95.5 kB ? eta -:--:--
   -------------------------------------- - 92.2/95.5 kB 1.7 MB/s eta 0:00:0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.23.4 requires anyio<4,>=3.1.0, but you have anyio 4.9.0 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [8]:
# main.py

from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import numpy as np

app = FastAPI()

# Load model
model = joblib.load("titanic_model.pkl")

# Define input schema
class Passenger(BaseModel):
    Pclass: int
    Sex: int     # 0 = male, 1 = female
    Age: float

@app.post("/predict")
def predict_survival(passenger: Passenger):
    data = np.array([[passenger.Pclass, passenger.Sex, passenger.Age]])
    prediction = model.predict(data)
    return {"survived": int(prediction[0])}


ImportError: cannot import name 'Doc' from 'typing_extensions' (C:\Users\Lenovo\anaconda3\Lib\site-packages\typing_extensions.py)